<a href="https://colab.research.google.com/github/taiki738/diffusers/blob/main/diffusers_LoRA_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# 1. Setup & Environment
# ==========================================
import os
from google.colab import drive

# Google Driveのマウント
drive.mount('/content/drive')

# 設定: リポジトリのパス
REPO_DIR = "/content/drive/MyDrive/github/diffusers"

# ディレクトリがない場合はクローン、ある場合は更新
if not os.path.exists(REPO_DIR):
    print(f"📥 Cloning repository to {REPO_DIR}...")
    !git clone https://github.com/taiki738/diffusers.git {REPO_DIR}
else:
    print(f"🔄 Updating repository in {REPO_DIR}...")
    os.chdir(REPO_DIR)
    !git pull origin main

# リポジトリディレクトリへ移動
os.chdir(REPO_DIR)
print(f"📂 Current directory: {os.getcwd()}")

# 依存ライブラリのインストール
print("⏳ Installing dependencies...")
!pip install -e .
!pip install -q accelerate transformers ftfy xformers

print("✅ Setup complete.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔄 Updating repository in /content/drive/MyDrive/github/diffusers...
From https://github.com/taiki738/diffusers
 * branch                main       -> FETCH_HEAD
Already up to date.
📂 Current directory: /content/drive/MyDrive/github/diffusers
⏳ Installing dependencies...
Obtaining file:///content/drive/MyDrive/github/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.37.0.dev0-0.editable-py3-none-any.whl size=11392 sha256=c42b8a613857bbcbe11a27d9620a2550eaece36320f90274a3333dd8e3d03478
  Stored in directory: /tmp/pip-ephem-wheel-cache-ioweuw8c/wheels/cb/0c/d2/eee942993f14cc21e8ffdade541

In [2]:
# ==========================================
# 2. Dataset Preparation
# ==========================================
# 設定: データセットのアーカイブパスと展開先
DATA_ARCHIVE_PATH = "/content/drive/MyDrive/github/latent-diffusion/data/ffhq_survey.tar.gz"
EXTRACT_DIR = "/content/ffhq_for_survey"

print(f"⏳ Extracting dataset from {DATA_ARCHIVE_PATH} to {EXTRACT_DIR}...")

# 既存データのクリーンアップ
!rm -rf {EXTRACT_DIR}

# アーカイブの解凍 (実体として展開)
!tar -xhzf "{DATA_ARCHIVE_PATH}" -C /content/

# メタデータ (metadata.jsonl) の生成
# push済みのリポジトリ内スクリプトを使用
print("⏳ Generating metadata...")
!python scripts/prepare_lora_colab.py --dataset_root {EXTRACT_DIR}

print("✅ Dataset ready.")

⏳ Extracting dataset from /content/drive/MyDrive/github/latent-diffusion/data/ffhq_survey.tar.gz to /content/ffhq_for_survey...
⏳ Generating metadata...
Scanning directory: /content/ffhq_for_survey
  - male/OK_4.0: 76 images found.
  - male/not-OK_2.0: 190 images found.
  - female/OK_4.0: 131 images found.
  - female/not-OK_2.0: 155 images found.

Success! Created 552 entries in /content/ffhq_for_survey/metadata.jsonl
You can now start training with --train_data_dir pointing to the dataset_root.
✅ Dataset ready.


In [ ]:
# ==========================================
# 3. Start LoRA Training
# ==========================================
# 設定: 学習結果の出力先 (Drive上推奨)
OUTPUT_DIR = "/content/drive/MyDrive/github/diffusers/outputs/lora_male_v1"

print(f"🚀 Starting training... Output will be saved to: {OUTPUT_DIR}")

# 学習コマンドの実行
!accelerate launch examples/text_to_image/train_text_to_image_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --train_data_dir="{EXTRACT_DIR}" \
  --resolution=512 \
  --random_flip \
  --train_batch_size=4 \
  --num_train_epochs=100 \
  --checkpointing_steps=500 \
  --learning_rate=1e-04 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --seed=42 \
  --rank=4 \
  --output_dir="{OUTPUT_DIR}" \
  --validation_prompt="a photo of a male face, high score impression" \
  --report_to="tensorboard" \
  --resume_from_checkpoint="latest"

print("🎉 Training process finished!")

🚀 Starting training... Output will be saved to: /content/drive/MyDrive/github/diffusers/outputs/lora_male_v1
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2026-01-13 02:55:53.927905: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-13 02:55:53.944884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E000